<a href="https://colab.research.google.com/github/nekotanku/colab/blob/master/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.utils.data import Dataset
torch.cuda.is_available()
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
if os.path.exists('/content/train_images/')==False:
    !unzip "/content/drive/My Drive/signate_img_labeling/train_images.zip"
else:
    pass

In [3]:
if os.path.exists('/content/train_images/')==False:
   !unzip "/content/drive/My Drive/signate_img_labeling/test_images.zip"
else:
    pass

In [4]:
train_master = pd.read_table('/content/drive/My Drive/signate_img_labeling/train_master.tsv')

In [5]:
train_master.head(10)

,file_name,label_id
0,train_0.jpg,1
1,train_1.jpg,5
2,train_2.jpg,1
3,train_3.jpg,6
4,train_4.jpg,3
5,train_5.jpg,9
6,train_6.jpg,7
7,train_7.jpg,4
8,train_8.jpg,5
9,train_9.jpg,8


In [6]:
#Datasetを継承
class Create_Datasets(Dataset):
     #初期化
    
    def __init__(self, dir_name, tsv_file, data_transform):
        self.dir_name = '/content/'
        self.tsv_file = 'drive/My Drive/signate_img_labeling/train_master.tsv'
        self.df = pd.read_table(self.dir_name + tsv_file)
        self.data_transform = data_transform
    
    #データの個数を返す
    def __len__(self):
        return len(self.df)
    
    #返却された数値分のiが処理される
    def __getitem__(self, i):
        file = self.df['file_name'][i]
        label = np.array(self.df['label_id'][i])
        image = Image.open(self.dir_name+ "train_images/" + file)
        image = self.data_transform(image)

        return image, label


In [7]:
data_transfrom = transforms.Compose([
                                    transforms.Resize(32*32),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.5,0.5,0.5], [0.5, 0.5, 0.5])
])

In [8]:
#create_datasetsのクラスをインスタンス化
class Mynomalize:
    def __call__(self, image):
        shape = image.shape
        image = (image - np.mean(image)) / np.std(image)*16+64
        return image

In [9]:
imgDataset = Create_Datasets('/content/', 'drive/My Drive/signate_img_labeling/train_master.tsv', data_transfrom)

In [ ]:
train_data, test_data = train_test_split(imgDataset, test_size=0.2)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=True)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(16*37*37, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), 16*37*37)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image, label = Variable(image), Variable(label)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(image), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data[0]))


def test():
    model.eval()

    for (image, label) in test_loader:
        image, label = Variable(image.float(), volatile=True), Variable(label)
        output = model(image)
        test_loss += criterion(output, label).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(label.data.view_as(pred)).long().cpu().sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
#学習
for epoch in range(1, 1000 + 1):
    train(epoch)
    test()
